In [3]:
import pandas as pd
# https://www.kaggle.com/c/fake-news/data?select=train.csv(DATASET)
data=pd.read_csv("train.csv")
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [4]:
data=data.dropna()
data.reset_index(inplace=True)

In [5]:
X = data.drop("label",axis=1)
y = data["label"]

In [6]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # (word not in stopwords) and stem(word)
  review = ' '.join(review)
  corpus.append(review)

In [12]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 


In [13]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  516 1627 1519]
 [   0    0    0 ... 1174 3028 3932]
 [   0    0    0 ... 3250 1234 3433]
 ...
 [   0    0    0 ... 3219 1575 3940]
 [   0    0    0 ... 4382 1828 3745]
 [   0    0    0 ... 4920 4162 3521]]


In [14]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [20]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 14s 51ms/step - loss: 0.3301 - accuracy: 0.8431 - val_loss: 0.2000 - val_accuracy: 0.9128
Epoch 2/10
192/192 [==============================] - 9s 49ms/step - loss: 0.1389 - accuracy: 0.9442 - val_loss: 0.2088 - val_accuracy: 0.9188
Epoch 3/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0904 - accuracy: 0.9665 - val_loss: 0.2115 - val_accuracy: 0.9176
Epoch 4/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0596 - accuracy: 0.9785 - val_loss: 0.2775 - val_accuracy: 0.9185
Epoch 5/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0387 - accuracy: 0.9877 - val_loss: 0.3290 - val_accuracy: 0.9150
Epoch 6/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0244 - accuracy: 0.9922 - val_loss: 0.3510 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0124 - accuracy: 0.9964 - val_loss: 0.4618 - val_accuracy: 0.915

In [21]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

189/189 [==============================] - 3s 11ms/step


array([[3131,  288],
       [ 254, 2362]], dtype=int64)

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9101905550952776